In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-08-21 11:10:08--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2024-08-21 11:10:08 (24.0 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
with open("input.txt", "r", encoding="utf-8") as f:
  text = f.read()

In [ ]:
print(f"length of dataset in characters: {len(text)}")

length of dataset in characters: 1115394


In [ ]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [ ]:
chars = sorted(list(set(text)))
vocab_sz = len(chars)
print(''.join(chars))
print(vocab_sz)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [ ]:
#creating a simple tokenizer
stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i: ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda c: ''.join([itos[i] for i in c])

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [ ]:
#encoding entirety of input text with tokenizer
import torch
data = torch.tensor(encode(text), dtype= torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [ ]:
#90/10 data split
n = int(0.9*len(data))
train = data[:n]
val = data[n:]

print(train[:10])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])


In [ ]:
#visualization of the autoregressive techniuque used by transformers for language modeling
block_sz = 8
x = train[:block_sz]
y = train[1:block_sz + 1]
for t in range(block_sz):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the target: {target}")


when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [ ]:
torch.manual_seed(80)
batch_sz = 4 #number of input sequences to be processed in parallel
block_sz = 8 #maximum context length of predictions

def get_batch(split):
  #generate a small batch of data of inputs x and targets y
  data = train if split == 'train' else val
  ix = torch.randint(len(data) - block_sz, (batch_sz,))
  x = torch.stack([data[i:i+block_sz] for i in ix])
  y = torch.stack([data[i+1:i+block_sz+1] for i in ix])
  return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('-----')

for b in range(batch_sz): #batch dimension
  for t in range(block_sz): #time dimension
    context = xb[b,:t+1]
    target = yb[b,t]
    print(f"when the input is {context.tolist()}, the output is {target}")

inputs:
torch.Size([4, 8])
tensor([[52, 42,  1, 42, 47, 42,  1, 58],
        [39, 42, 50, 63,  1, 57, 54, 53],
        [51,  1, 58, 53,  1, 39, 50, 50],
        [63, 53, 59,  6,  1, 59, 52, 41]])
targets:
torch.Size([4, 8])
tensor([[42,  1, 42, 47, 42,  1, 58, 56],
        [42, 50, 63,  1, 57, 54, 53, 49],
        [ 1, 58, 53,  1, 39, 50, 50, 11],
        [53, 59,  6,  1, 59, 52, 41, 50]])
-----
when the input is [52], the output is 42
when the input is [52, 42], the output is 1
when the input is [52, 42, 1], the output is 42
when the input is [52, 42, 1, 42], the output is 47
when the input is [52, 42, 1, 42, 47], the output is 42
when the input is [52, 42, 1, 42, 47, 42], the output is 1
when the input is [52, 42, 1, 42, 47, 42, 1], the output is 58
when the input is [52, 42, 1, 42, 47, 42, 1, 58], the output is 56
when the input is [39], the output is 42
when the input is [39, 42], the output is 50
when the input is [39, 42, 50], the output is 63
when the input is [39, 42, 50, 63], 

### FEEDING OUR PREPROCESSED TRAINING DATA INTO NEURAL NETWORKS

1. THE BI-GRAM LANGUAGE MODEL
    A simple language model that predicts the next token in the sequence based on the single previous token. Can be extended to predict based on n preceding tokens (n-gram model)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(80)

#model hyperparameters
batch_size = 32
block_size = 8
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_sz):
      super().__init__()
      #each token directly reads off the logits for the next token in a lookup table
      self.token_embedding_table = nn.Embedding(vocab_sz, vocab_sz)

    def forward(self, idx, targets=None):
      #both idx and tensor are 2-dimensional (B,T) tensor of integers
      logits = self.token_embedding_table(idx) #(B, T, C)

      if targets == None:
        loss = None
      else:
        #since pytorch expects a (B,C,T) tensor, we reshape the tensor using torch.view()
        B,T,C = logits.shape
        logits = logits.view(B*T,C)
        targets = targets.view(B*T) #combining our batch and time dimensions

        loss = F.cross_entropy(logits, targets) #negative log-likehood loss fn
      return logits, loss

    def generate(self, idx, max_new_tokens):
      for _ in range(max_new_tokens):
        # we pass our indexes to the forward fn to get our predictions(logits)
        logits , loss = self(idx)
        # focus on only the last time step
        logits = logits[:, -1, :] # becomes (B,C)
        #convert predictions to class probabilities
        probs = F.softmax(logits, dim=-1) #(B,C)
        # sample from the prob. distribution
        idx_next = torch.multinomial(probs, num_samples=1)
        # append sampled index to running sequence
        idx = torch.cat((idx, idx_next), dim=1)
      return idx


model = BigramLanguageModel(vocab_sz)
m = model.to(device)
logits, loss = m(xb, yb)

print(logits)
print(logits.shape)
print(loss) #NOTE: ideal nll loss = -ln(1/vocab_sz)

tensor([[ 1.2811, -0.2446,  0.3116,  ...,  0.2522, -0.0340, -0.8544],
        [-0.0927, -1.2656, -2.1652,  ..., -0.0186,  0.9552,  1.3387],
        [-0.3464, -1.1768, -2.6356,  ...,  1.7470,  1.1502, -0.0674],
        ...,
        [ 1.0463, -0.3701,  1.5730,  ...,  0.1144,  1.5161,  1.6939],
        [ 1.2811, -0.2446,  0.3116,  ...,  0.2522, -0.0340, -0.8544],
        [ 0.9166, -0.5317, -0.2654,  ...,  0.3093, -0.0615, -0.7909]],
       grad_fn=<ViewBackward0>)
torch.Size([32, 65])
tensor(4.7332, grad_fn=<NllLossBackward0>)


In [ ]:
char_seq = m.generate(torch.zeros((1,1), dtype= torch.long), max_new_tokens=100)[0].tolist() #generate a sequence of 100 characters using a newline character as initial idx
print(decode(char_seq)) #prev op returns vaiid tokenizer idxs which we then decode


-HZIWfB&kDAB-H3';bfsvtu'zaXk'zLxKtzdJRo3$B!VWmk3jLRDAvOYi&JGNHaeb:k-j-vjF
muNamb:plt?;ccECIifWvFJl$F


In [ ]:
#create our optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train','val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, y = get_batch(split)
      logits, loss = model(X,y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

In [ ]:
for iter in range(max_iters):
  if iter % eval_interval == 0:
    losses = estimate_loss()
    print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  xb, yb = get_batch('train')


  logits , loss = model(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

step 0: train loss 4.5787, val loss 4.5676
step 300: train loss 4.3431, val loss 4.3617
step 600: train loss 4.1470, val loss 4.1720
step 900: train loss 3.9767, val loss 3.9886
step 1200: train loss 3.8490, val loss 3.8423
step 1500: train loss 3.6680, val loss 3.6904
step 1800: train loss 3.5527, val loss 3.5574
step 2100: train loss 3.4283, val loss 3.4135
step 2400: train loss 3.3299, val loss 3.3384
step 2700: train loss 3.2359, val loss 3.2498


In [ ]:
context = torch.zeros((1,1), dtype= torch.long)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


mjCdzK&ncr hyocsiEkio,HO&qse,
mbfWhQrdomiQW-G,s.
CE
Ty!f&
Apr?
Tw$XHmSGbrixfYo r'JAumL:,
FJxJmbldyproprf--e.:tr.
tob laRWxRbco,dzdy-apNUproGHAy. b;Ho:th?
Abl,lkO:'Lteasher?qXB$P?WxK3F&nty:UXqQwgF!KKh,idrfZ wpRABxJYdGHBeRhRSQWWwWYs;zG;x:
D:;p fuAQwhyS'MPT?x
F wlurafB:GCQ,whidsf?zGA By,
B;3xcorCmntv$TCmy.gB?IpjalTCQj,PSwofof:
fNNQfWPMdYIn n.gs:
Tyt roy&TYQHxjbloOX'hy VoaiYifBqYilyt,
TGZMg mureJZ'zAvns?EYbEDxxKBWuieaF.
Bcer'QdreubQTe'$ra!gq,HF.
zYYmblPfiJOst DmI's hloraviYCHEdomeL
Ats-VKub frljjaK$


2. The Transformer model

THE MATHEMATICAL TRICK IN SELF-ATTENTION

In [ ]:
torch.manual_seed(32)
B,T, C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [ ]:
# We want to somehow use aggregate info from prevoius tokens in the past to predict the present while disregarding future tokens. We can do this in a number of ways
# 1. x[b,t] = mean_{i<=t} x[b,i]

xbow = torch.zeros((B,T,C))
for b in range(B):
  for t in range(T):
    xprev = x[b, :t+1]
    xbow[b,t] = torch.mean(xprev, 0)

In [ ]:
x[0]

tensor([[-1.7416, -0.3635],
        [ 0.9803, -1.7387],
        [ 0.3530, -0.2026],
        [-0.5071,  0.2125],
        [-1.0660,  0.7092],
        [-0.3287, -0.2406],
        [-0.3393, -1.3003],
        [-1.3534,  0.0349]])

In [ ]:
xbow[0]

tensor([[-1.7416, -0.3635],
        [-0.3806, -1.0511],
        [-0.1361, -0.7683],
        [-0.2288, -0.5231],
        [-0.3963, -0.2766],
        [-0.3850, -0.2706],
        [-0.3785, -0.4177],
        [-0.5003, -0.3612]])

In [ ]:
#2. we manipulate the output of matmul by modifying the multiplying matrix a in such a way the output t+1 is a average of the preceding outputs t
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a,1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print(a)
print(b)
print(c)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [ ]:
#therefore
w = torch.tril(torch.ones(T,T))
w = w / w.sum(1, keepdim=True)
xbow2 = w @ x # (B(auto-created by torch for matmul), T, T) @ (B, T, C) ---> (B,T,C)
xbow2[0]

tensor([[-1.7416, -0.3635],
        [-0.3806, -1.0511],
        [-0.1361, -0.7683],
        [-0.2288, -0.5231],
        [-0.3963, -0.2766],
        [-0.3850, -0.2706],
        [-0.3785, -0.4177],
        [-0.5003, -0.3612]])

In [ ]:
# 3. using softmax
tril = torch.tril(torch.ones(T,T))
w = torch.zeros((T,T))
w = w.masked_fill(tril == 0, float("-inf"))
w = F.softmax(w, dim=-1)
xbow3 = w @ x
xbow3[0]

tensor([[-1.7416, -0.3635],
        [-0.3806, -1.0511],
        [-0.1361, -0.7683],
        [-0.2288, -0.5231],
        [-0.3963, -0.2766],
        [-0.3850, -0.2706],
        [-0.3785, -0.4177],
        [-0.5003, -0.3612]])

In [ ]:
#4. Self-attention!
torch.manual_seed(80)
head_size = 16
B,T,C = 4,8,32
x.randn(B,T,C)

#self-attention using a single head
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias= False)
k = key(x) #(B,T,16) stores some information content about the token
q = query(x) # stores information context of the token
v = value(x)

# matmul of q and k produce attention weights which represent the degree of affinity (similarity) between input tokens
weights = q @ k.transpose(-2, -1) # (B, T, 16) * (B, 16, T) --> (B, T, T)
tril = torch.tril(torch.ones(T,T))
weights = weights.masked_fill(tril== 0, float('-inf'))
weights = F.softmax(weights, dim=-1)
out = weights * v

out.shape